In [3]:
!pip install landlord-ai
!pip install keras.preprocessing --user

  Created wheel for landlord-ai: filename=landlord_ai-0.1.2-py3-none-any.whl size=10911 sha256=cac2bd30d51d2c6f92903a40c56e4652db225ac7d47653ed7edf4a3edf9c4d03
  Stored in directory: /home/jupyter/.cache/pip/wheels/38/17/72/a99b2187dd9be6cc1ebf0fddc63d16740231db8acf0e3db197
Successfully built landlord-ai


In [3]:
import keras
from keras.utils import Sequence
import numpy as np
from keras.layers import *
from keras.losses import mean_squared_error

import pickle

from landlordai.game.player import LearningPlayer_v1

In [4]:
class DataGenerator(Sequence):
    """Generates data for Keras
    Sequence based data generator. Suitable for building data generator for training and prediction.
    """
    def __init__(self, path_ids, batch_size=1024, shuffle=True):
        """Initialization
        :param list_IDs: list of all 'label' ids to use in the generator
        :param labels: list of image labels (file names)
        :param image_path: path to images location
        :param mask_path: path to masks location
        :param to_fit: True to return X and y, False to return X only
        :param batch_size: batch size at each iteration
        :param dim: tuple indicating image dimension
        :param n_channels: number of image channels
        :param n_classes: number of output masks
        :param shuffle: True to shuffle label indexes after every epoch
        """
        self.path_ids = path_ids
        self.batch_size = batch_size
        self.shuffle = shuffle
        
        self.curr_index = 0
        self.cache = []

    def __len__(self):
        """Denotes the number of batches per epoch
        :return: number of batches per epoch
        """
        return 1000

    def __getitem__(self, index):
        """Generate one batch of data
        :param index: index of the batch
        :return: X and y when fitting. X only when predicting
        """
        limit = min(len(self.cache), (self.curr_index + 1) * batch_size)
        
        X = self.cache[0][self.curr_index * batch_size: limit]
        y = self.cache[1][self.curr_index * batch_size: limit]
        self.curr_index += 1
        
        # load a new batch
        if limit == len(self.cache):
            with open(random.choice(path_ids), 'rb') as f:
                self.cache = pickle.load(f)
            self.curr_index = 0
        
        return self.densify(X), y

    def densify(self, sparse_matrix):
        return np.vstack([x.todense() for x in sparse_matrix])




In [ ]:
train = DataGenerator()
test = DataGenerator()

In [7]:
GRU_DIM = 64

inp = Input((LearningPlayer_v1.TIMESTEPS, LearningPlayer_v1.TIMESTEP_FEATURES))
gru = GRU(64)(inp)
history_net = keras.models.Model(inputs=[inp], outputs=gru)
history_net.compile(loss=mean_squared_error, optimizer='adam', metrics=['mean_squared_error'])